In [ ]:
# Setup: Initialize variable to trigger fallback
sales_data_info = None

In [42]:
# This cell acts as a fallback if the primary MCP analysis in the previous cell fails.if 'sales_data_info' not in locals() or not sales_data_info:    print("⚠️ MCP analysis failed or returned no data. Initiating local fallback...")    try:        import pandas as pd        from pathlib import Path        import json        # FIXED: Use CSV file instead of encrypted Excel file        csv_path = Path("./sample-data/csv/sales_performance.csv")        if not csv_path.exists():            print(f"❌ Fallback failed: CSV file not found at {csv_path.resolve()}")        else:            print(f"✅ Found local CSV file: {csv_path.resolve()}")            # Read the CSV file using pandas            df = pd.read_csv(csv_path)            # Generate a structure summary similar to the MCP output            structure = {                "file_name": csv_path.name,                "columns": df.columns.tolist(),                "row_count": len(df),                "column_types": {col: str(dtype) for col, dtype in df.dtypes.items()},                "sample_data": df.head(3).to_dict('records')            }            # Create a formatted string summary            summary_lines = [                f"File Name: {structure['file_name']}",                f"Total Rows: {structure['row_count']}",                f"Columns ({len(structure['columns'])}):"            ]            for col, dtype in structure['column_types'].items():                summary_lines.append(f"  - {col} (Type: {dtype})")            summary_lines.append("\nSample Data (First 3 Rows):")            for i, row in enumerate(structure['sample_data'], 1):                summary_lines.append(f"  Row {i}:")                for key, val in row.items():                    summary_lines.append(f"    {key}: {val}")            # Store the summary in the required variable            sales_data_info = "\n".join(summary_lines)            print("\n" + "="*80)            print("✅ LOCAL FALLBACK ANALYSIS COMPLETE")            print("="*80)            print(sales_data_info)    except Exception as e:        print(f"❌ An error occurred during local fallback analysis: {e}")        import traceback        traceback.print_exc()else:    print("✅ MCP analysis was successful. Skipping local fallback.")

⚠️ MCP analysis failed or returned no data. Initiating local fallback...
 Found local Excel file: C:\Users\lproux\OneDrive - Microsoft\bkp\Documents\GitHub\MCP-servers-internalMSFT-and-external\AI-Gateway\labs\master-lab\sample-data\excel\sales_performance.xlsx
 An error occurred during local fallback analysis: File is not a zip file


Traceback (most recent call last):
  File "C:\Users\lproux\AppData\Local\Temp\ipykernel_9408\3412618939.py", line 20, in <module>
    df = pd.read_excel(excel_path, engine='openpyxl')
  File "c:\Users\lproux\OneDrive - Microsoft\bkp\Documents\GitHub\.venv_win_backup\Lib\site-packages\pandas\io\excel\_base.py", line 495, in read_excel
    io = ExcelFile(
        io,
    ...<2 lines>...
        engine_kwargs=engine_kwargs,
    )
  File "c:\Users\lproux\OneDrive - Microsoft\bkp\Documents\GitHub\.venv_win_backup\Lib\site-packages\pandas\io\excel\_base.py", line 1567, in __init__
    self._reader = self._engines[engine](
                   ~~~~~~~~~~~~~~~~~~~~~^
        self._io,
        ^^^^^^^^^
        storage_options=storage_options,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        engine_kwargs=engine_kwargs,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\lproux\OneDrive - Microsoft\bkp\Documents\GitHub\.venv_win_backup\Lib\site-packages\pandas\io\excel\_openpyxl.py"

In [ ]:
print('\n' + '='*80 + '\nTEST 1 COMPLETE\n' + '='*80)

In [43]:
# Exercise 2.3: Azure Cost Analysis via Local Pandasprint("💰 Azure Cost Analysis via Local Pandas + Azure OpenAI")print("=" * 80)from pathlib import Pathimport pandas as pdtry:    # FIXED: Use CSV file instead of encrypted Excel file    cost_file_path = Path("./sample-data/csv/azure_resource_costs.csv")    if not cost_file_path.exists():        raise FileNotFoundError(f"Cost file not found at '{cost_file_path.resolve()}'")    print(f"✅ Reading cost file locally: {cost_file_path.name}")        # Read CSV file using pandas    df = pd.read_csv(cost_file_path)        print(f"✅ File loaded successfully: {len(df)} rows, {len(df.columns)} columns")        # Display columns    print(f"\n📊 Columns: {df.columns.tolist()}")        # Preview data    print("\n👀 Preview (first 3 rows):")    print(df.head(3).to_string())    # Determine column names (case-insensitive search)    group_by_col = None    metric_col = None        # Look for ResourceType or ResourceName for grouping    for col in df.columns:        if 'resourcetype' in col.lower() or ('resource' in col.lower() and 'type' in col.lower()):            group_by_col = col        if 'cost' in col.lower():            metric_col = col        if not group_by_col:        # Try ResourceName as fallback        for col in df.columns:            if 'resourcename' in col.lower() or 'resource' in col.lower():                group_by_col = col                break        if not group_by_col or not metric_col:        print(f"⚠️ Warning: Could not auto-detect columns. Available: {df.columns.tolist()}")        # Use defaults if detection fails        group_by_col = group_by_col or df.columns[0]        metric_col = metric_col or df.columns[1] if len(df.columns) > 1 else df.columns[0]    print(f"\n📈 Analyzing costs with group_by='{group_by_col}' and metric='{metric_col}'")    # Group by service and sum costs    cost_breakdown = df.groupby(group_by_col)[metric_col].agg(['sum', 'mean', 'count']).reset_index()    cost_breakdown.columns = ['ServiceName', 'Total', 'Average', 'Count']    cost_breakdown = cost_breakdown.sort_values('Total', ascending=False)        # Calculate totals    total_cost = cost_breakdown['Total'].sum()    # Assuming the data represents daily costs, calculate a monthly projection    monthly_projection = total_cost * 30    print("\n💵 Cost Analysis Summary:")    print("=" * 80)    print("📊 Cost Breakdown by Service Name:")    for idx, row in cost_breakdown.iterrows():        print(f"  - {row['ServiceName']}: ${row['Total']:,.2f} (Avg: ${row['Average']:,.2f}, Count: {row['Count']})")    print(f"\n💰 Total Daily Cost: ${total_cost:,.2f}")    print(f"📅 Projected Monthly Cost: ${monthly_projection:,.2f}")    # Create a compact summary for AI analysis    cost_data_info = f"""Total Daily Cost: ${total_cost:,.2f}Projected Monthly Cost: ${monthly_projection:,.2f}Top Service: {cost_breakdown.iloc[0]['ServiceName']} (${cost_breakdown.iloc[0]['Total']:,.2f})Number of Services: {len(cost_breakdown)}"""    print("\n📋 Compact cost_data_info for AI prompts:")    print(cost_data_info)except Exception as e:    print(f"❌ ERROR (Cost Analysis): {e}")    print("🔍 Troubleshooting:")    print("  • Ensure the cost file exists at './sample-data/csv/azure_resource_costs.csv'")    print("  • Verify the file is a valid CSV format")    print("  • Check that pandas is installed: pip install pandas")    import traceback    traceback.print_exc()

 Azure Cost Analysis via Local Pandas + Azure OpenAI
 Reading cost file locally: azure_resource_costs.xlsx
 ERROR (Cost Analysis): File is not a zip file
 Troubleshooting:
  • Ensure the cost file exists at './sample-data/excel/azure_resource_costs.xlsx'
  • Verify the file is a valid .xlsx format
  • Check that openpyxl is installed: pip install openpyxl


Traceback (most recent call last):
  File "C:\Users\lproux\AppData\Local\Temp\ipykernel_9408\4037427367.py", line 17, in <module>
    df = pd.read_excel(cost_file_path, engine='openpyxl')
  File "c:\Users\lproux\OneDrive - Microsoft\bkp\Documents\GitHub\.venv_win_backup\Lib\site-packages\pandas\io\excel\_base.py", line 495, in read_excel
    io = ExcelFile(
        io,
    ...<2 lines>...
        engine_kwargs=engine_kwargs,
    )
  File "c:\Users\lproux\OneDrive - Microsoft\bkp\Documents\GitHub\.venv_win_backup\Lib\site-packages\pandas\io\excel\_base.py", line 1567, in __init__
    self._reader = self._engines[engine](
                   ~~~~~~~~~~~~~~~~~~~~~^
        self._io,
        ^^^^^^^^^
        storage_options=storage_options,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        engine_kwargs=engine_kwargs,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\lproux\OneDrive - Microsoft\bkp\Documents\GitHub\.venv_win_backup\Lib\site-packages\pandas\io\excel\_openpyxl